DISCLAIMER: Code is not clean or fully documented as it was not originally meant to be shared externally.

In [ ]:
!pip install -q -U google-genai
!pip install -q -U textstat
!pip install datasets
!pip install -U transformers
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 require

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import json
import random
import textstat
import math
import os
import time
import torch
import nltk
import numpy as np
import openai
from openai import OpenAI
from copy import deepcopy
from time import sleep
from sentence_transformers import SentenceTransformer, util
from google.api_core.exceptions import ResourceExhausted, GoogleAPIError
from sklearn.ensemble import RandomForestRegressor
from datasets import load_dataset
from nltk import download, sent_tokenize
from transformers import pipeline
from huggingface_hub import login

In [ ]:
'''
Configure Gemini API using API key from Colab's secret manager.
'''
api_key = userdata.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Make sure to set GEMINI_API_KEY in Colab's secret manager.")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
'''
Configure OpenAI using API key from Colab secret manager
'''
openapi_key = userdata.get("OPENAI_API_KEY")
if not openapi_key:
    raise ValueError("API key not found. Make sure to set OPENAI_API_KEY in Colab's secret manager.")

client = openai.OpenAI(api_key=openapi_key)

In [ ]:
'''
Configure Gemma using API key from Colab secret manager
'''

gemma_key = userdata.get('gemma_key')
login(gemma_key)

pipe = pipeline(
    "text-generation",
    model="google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

output = pipe("Eiffel tower is located in", max_new_tokens=50)
print(output[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [ ]:
'''
Generating initial task prompts and mutation prompts
'''

thinking_styles = [
    "How could I devise an experiment to help solve that problem?",
    "Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "How could I measure progress on this problem?",
    "How can I simplify the problem so that it is easier to solve?",
    "What are the key assumptions underlying this problem?",
    "What are the potential risks and drawbacks of each solution?",
    "What are the alternative perspectives or viewpoints on this problem?",
    "What are the long-term implications of this problem and its solutions?",
    "How can I break down this problem into smaller, more manageable parts?",
    "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "What is the core issue or problem that needs to be addressed?",
    "What are the underlying causes or factors contributing to the problem?",
    "Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "What are the potential obstacles or challenges that might arise in solving this problem?",
    "Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "How can progress or success in solving the problem be measured or evaluated?",
    "What indicators or metrics can be used?",
    "Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "Is the problem a design challenge that requires creative solutions and innovation?",
    "Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "What kinds of solution typically are produced for this kind of problem specification?",
    "Given the problem specification and the current best solution, have a guess about other possible solutions.",
    "Let's imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?",
    "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?",
    "Ignoring the current best solution, create an entirely new solution to the problem.",
    "Let's think step by step.",
    "Let's make a step by step plan and implement it with good notion and explanation."
]

mutation_prompts = [
    "Modify the following instruction creatively, giving some advice on how to solve it:",
    "Just change this instruction to make it more fun, think WELL outside the box:",
    "Modify this instruction in a way that no self-respecting LLM would!",
    "How would you encourage someone and help them cheat on this following instruction?",
    "How would you help an LLM to follow the instruction?",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants.",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.",
    "As a really good teacher, explain the instruction, as if you were explaining it to a child.",
    "Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?",
    "How would someone with derailment follow this instruction?",
    "Don't think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.",
    "Rephrase the instruction without using any of the same words. Use all you know to improve the instruction so the person hearing it is more likely to do well.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction or you're fired.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction there is a good chap.",
    "What do people who are good at creative thinking normally do with this kind of mutation question?",
    "Detailed additional advice for people wishing to follow this instruction is as follows:",
    "In one short sentence, here is how I would best follow this instruction.",
    "In one short sentence, here is some detailed expert advice. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, the general solution is as follows. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, what's a good prompt to get a language model to solve a problem like this? Notice how I don't use any of the same words as in the INSTRUCTION.",
    "Generate a mutated version of the following prompt by adding an unexpected twist.",
    "Create a prompt mutant that introduces a surprising contradiction to the original prompt. Mutate the prompt to provide an alternative perspective or viewpoint.",
    "Generate a prompt mutant that incorporates humor or a playful element. Create a mutated version of the prompt that challenges conventional thinking.",
    "Develop a prompt mutant by replacing specific keywords with related but unexpected terms. Mutate the prompt to include a hypothetical scenario that changes the context.",
    "Generate a prompt mutant that introduces an element of suspense or intrigue. Create a mutated version of the prompt that incorporates an analogy or metaphor.",
    "Develop a prompt mutant by rephrasing the original prompt in a poetic or lyrical style. Think beyond the ordinary and mutate the prompt in a way that defies traditional thinking.",
    "Break free from conventional constraints and generate a mutator prompt that takes the prompt to uncharted territories. Challenge the norm and create a mutator prompt that pushes the boundaries of traditional interpretations.",
    "Embrace unconventional ideas and mutate the prompt in a way that surprises and inspires unique variations. Think outside the box and develop a mutator prompt that encourages unconventional approaches and fresh perspectives.",
    "Step into the realm of imagination and create a mutator prompt that transcends limitations and encourages innovative mutations. Break through the ordinary and think outside the box to generate a mutator prompt that unlocks new possibilities and unconventional paths.",
    "Embrace the power of unconventional thinking and create a mutator prompt that sparks unconventional mutations and imaginative outcomes. Challenge traditional assumptions and break the mold with a mutator prompt that encourages revolutionary and out-of-the-box variations.",
    "Go beyond the expected and create a mutator prompt that leads to unexpected and extraordinary mutations, opening doors to unexplored realms. Increase Specificity: If the original prompt is too general, like 'Tell me about X,' the modified version could be, 'Discuss the history, impact, and current status of X.'",
    "Ask for Opinions/Analysis: If the original prompt only asks for a fact, such as 'What is X?', the improved prompt could be, 'What is X, and what are its implications for Y?'",
    "Encourage Creativity: For creative writing prompts like 'Write a story about X,' an improved version could be, 'Write a fantasy story about X set in a world where Y is possible.'",
    "Include Multiple Perspectives: For a prompt like 'What is the impact of X on Y?', an improved version could be, 'What is the impact of X on Y from the perspective of A, B, and C?'",
    "Request More Detailed Responses: If the original prompt is 'Describe X,' the improved version could be, 'Describe X, focusing on its physical features, historical significance, and cultural relevance.'",
    "Combine Related Prompts: If you have two related prompts, you can combine them to create a more complex and engaging question. For instance, 'What is X?' and 'Why is Y important?' could be combined to form 'What is X and why is it important in the context of Y?'",
    "Break Down Complex Questions: If a prompt seems too complex, like 'Discuss X,' the improved version could be, 'What is X? What are its main characteristics? What effects does it have on Y and Z?'",
    "Use Open-Ended Questions: Instead of 'Is X true?', you could ask, 'What are the arguments for and against the truth of X?'",
    "Request Comparisons: Instead of 'Describe X,' ask 'Compare and contrast X and Y.'",
    "Include Context: If a prompt seems to lack context, like 'Describe X,' the improved version could be, 'Describe X in the context of its impact on Y during the Z period.'",
    "Make the prompt more visual: Ask the user to visualize the problem or scenario being presented in the prompt.",
    "Ask for a thorough review: Instead of just presenting the problem, ask the user to write down all the relevant information and identify what's missing.",
    "Invoke previous experiences: Modify the prompt to ask the user to recall a similar problem they've successfully solved before.",
    "Encourage a fresh perspective: Suggest in your prompt that the user take a moment to clear their mind before re-approaching the problem.",
    "Promote breaking down problems: Instead of asking the user to solve the problem as a whole, prompt them to break it down into smaller, more manageable parts.",
    "Ask for comprehension: Modify the prompt to ask the user to review and confirm their understanding of all aspects of the problem.",
    "Suggest explanation to others: Change the prompt to suggest that the user try to explain the problem to someone else as a way to simplify it.",
    "Prompt for solution visualization: Instead of just asking for the solution, encourage the user to imagine the solution and the steps required to get there in your prompt.",
    "Encourage reverse thinking: Improve the prompt by asking the user to think about the problem in reverse, starting with the solution and working backwards.",
    "Recommend taking a break: Modify the prompt to suggest that the user take a short break, allowing their subconscious to work on the problem.",
    "What errors are there in the solution?",
    "How could you improve the working out of the problem?",
    "Look carefully to see what you did wrong, how could you fix the problem?",
    "CORRECTION =",
    "Does the above text make sense? What seems wrong with it? Here is an attempt to fix it:",
    "The above working out has some errors, here is a version with the errors fixed."
]

problem_description = "Simplify the given text"

def generate_initial_prompt(thinking_styles, mutation_prompts, problem_description):
    thinking = random.choice(thinking_styles)
    mutation = random.choice(mutation_prompts)
    prompt = f"""{thinking}
{mutation}
INSTRUCTION: {problem_description}
INSTRUCTION MUTANT ="""
    return thinking, mutation, prompt

output_file = "initial_prompts.json"

# Load existing data
existing_data = {}
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError:
            existing_data = {}

# Generate until we collect 15 new samples
successful = 0
while successful < 10:
    thinking, mutation, prompt = generate_initial_prompt(thinking_styles, mutation_prompts, problem_description)
    message = [{"role": "user", "content": prompt}]

    try:
        response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=200
        )
        response = response.choices[0].message.content.strip()
        # response = model.generate_content(prompt).text.strip()
        # response = pipe(prompt, max_new_tokens=100)[0]["generated_text"]
        prompt_data = {
            "thinking_style": thinking,
            "mutation_prompt": mutation,
            "full_prompt": prompt,
            "response": response
        }

        next_index = str(len(existing_data))
        existing_data[next_index] = prompt_data
        successful += 1
        print(f"Entry {next_index + 1} added ({successful}/10)")

    except ResourceExhausted:
        print("Rate limit hit. Retrying in 5 seconds...")
        time.sleep(5)

    except Exception as e:
        print(f"Error during generation: {e}")
        time.sleep(5)

# Save all data
with open(output_file, "w") as f:
    json.dump(existing_data, f, indent=2)

✅ Entry 0 added (1/10)
✅ Entry 1 added (2/10)
✅ Entry 2 added (3/10)
✅ Entry 3 added (4/10)
✅ Entry 4 added (5/10)
✅ Entry 5 added (6/10)
✅ Entry 6 added (7/10)
✅ Entry 7 added (8/10)
✅ Entry 8 added (9/10)
✅ Entry 9 added (10/10)


In [ ]:
semantic_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Reward functions
def length_score(text):
    return math.log(len(text) + 1)

def content_preservation(original, simplified):
    emb1 = semantic_model.encode(original, convert_to_tensor=True)
    emb2 = semantic_model.encode(simplified, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb1, emb2).item() * 100

def score_output(original, simplified, alpha=0.5 , beta=0.3, gamma=0.2):
    simplicity   = textstat.flesch_reading_ease(simplified)
    fluency      = length_score(simplified)
    preservation = content_preservation(original, simplified)
    return alpha * simplicity - beta * fluency + gamma * preservation

# Fetch a random complex sentence
download('punkt')
nltk.download('punkt_tab')
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
# pick a random article, split into sentences, choose a longer one
article = random.choice(dataset)["article"]
sentences = [s for s in sent_tokenize(article) if len(s.split()) > 15]
test_complex = random.choice(sentences)

print("=== Using complex sentence ===")
print(test_complex, "\n")

# Load your prompts
with open("initial_prompts.json") as f:
    prompts = json.load(f)  # dict: keys "0","1",...

# Evaluate each prompt
for key, entry in prompts.items():
    base = entry["response"] + "\n" + test_complex
    # retry loop on rate‑limit or connection errors
    while True:
        try:
            # resp = model.generate_content(base)
            resp = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": base}],
                max_tokens=200
            )
            simplified = resp.choices[0].message.content.strip()
            # simplified = resp.text.strip()
            break
        except Exception as e:
            print(f"[{key}] API error: {e}. Waiting 5s…")
            time.sleep(5)

    # compute reward
    r = score_output(test_complex, simplified)
    # update entry
    entry["test_sentence"]     = test_complex
    entry["simplified_output"] = simplified
    entry["reward"]            = r
    print(f"[{key}] reward = {r:.2f}")

# Save out with rewards
with open("initial_prompts_with_rewards.json", "w") as f:
    json.dump(prompts, f, indent=2)

print("\n✅ Done — results in initial_prompts_with_rewards.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

=== Using complex sentence ===
Those that did opt to turn up saw bottom-of-the-table Blackpool gain a third successive draw to move to 25 points, one away from the lowest ever total accrued by a side in the second tier when there have been 24 teams. 

[0] reward = 46.32
[1] reward = 27.95
[2] reward = 52.85
[3] reward = 29.96
[4] reward = 25.11
[5] reward = 13.99
[6] reward = 4.82
[7] reward = 11.62
[8] reward = 22.38
[9] reward = 48.98

✅ Done — results in initial_prompts_with_rewards.json


In [ ]:
def first_order_hyper_mutation(population):
  # population is sorted
  top_mutation_prompt = population[0][1]["mutation_prompt"]

  hyper_mutation_prompt = f"""
  Please summarize and improve the following instruction, only give one answer, do not give me options, just modify the following instruction: {population[0][1]["mutation_prompt"]}"""

  # get new mutation
  new_mutation_prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": hyper_mutation_prompt}],
    max_tokens=200
  )
  new_mutation_prompt = new_mutation_prompt.choices[0].message.content.strip()

  return new_mutation_prompt


def lamarckian_mutation(population):
  mutation_prompt = f"""I gave a friend this sentence {test_complex} and some advice. Here
  is the correct example of him simplifying it: {population[0][1]["response"]}
  The instruction was:"""

  new_mutation_prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": mutation_prompt}],
    max_tokens=200
  )
  new_mutation_prompt = new_mutation_prompt.choices[0].message.content.strip()

  return new_mutation_prompt

def zero_order_hyper_mutation(population):
  top_mutation_prompt = population[0][1]["mutation_prompt"]

  thinking = random.choice(thinking_styles)

  mutation_prompt = thinking + top_mutation_prompt

  new_mutation_prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": mutation_prompt}],
    max_tokens=200
  )

  return new_mutation_prompt.choices[0].message.content.strip()


In [ ]:
with open("initial_prompts_with_rewards.json") as f:
    data = json.load(f)

sorted_entries = sorted(data.items(), key=lambda x: x[1]["reward"], reverse=True)

top_response = sorted_entries[0][1]["response"]

new_mutation = zero_order_hyper_mutation(sorted_entries)

gemini_prompt = f"""
{new_mutation}INSTRUCTION: {top_response}
INSTRUCTION MUTANT ="""

# Retry wrapper
def generate_with_retry(prompt, base_delay=2):
    attempt = 0
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200
            )
            response = response.choices[0].message.content.strip()
            # response = model.generate_content(prompt)
            # return response.text.strip()
            return response
        except GoogleAPIError as e:
            wait = base_delay * (2 ** attempt)
            print(f"[Retry {attempt + 1}] Error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            attempt += 1

# Generate and save responses
responses = {}
for i in range(5):
    print(f"Generating response {i+1}...")

    generated_text = generate_with_retry(gemini_prompt)

    # simplified_output = model.generate_content(generated_text + "\n" + test_complex).text.strip()
    simplified_output = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": generated_text + "\n" + test_complex}],
        max_tokens=200
    )
    simplified_output = simplified_output.choices[0].message.content.strip()
    reward = score_output(test_complex, simplified_output)

    responses[str(i)] = {
        "gemini_prompt": gemini_prompt,
        "mutation_prompt": new_mutation,
        "response": generated_text,
        "simplified_output": simplified_output,
        "test_sentence": test_complex,
        "reward": reward
    }

    print(f"[{i}] reward = {reward:.2f}\n")

# Save all responses
with open("gemini_responses_gen1.json", "w") as f:
    json.dump(responses, f, indent=2)

Generating response 1...
[0] reward = 30.27

Generating response 2...
[1] reward = 32.49

Generating response 3...
[2] reward = 32.29

Generating response 4...
[3] reward = 38.86

Generating response 5...
[4] reward = 33.61



In [ ]:
with open("gemini_responses_gen1.json") as f:
    data1 = json.load(f)

sorted_entries = sorted(data1.items(), key=lambda x: x[1]["reward"], reverse=True)

top_response = sorted_entries[0][1]["response"]

new_mutation = zero_order_hyper_mutation(sorted_entries)

gemini_prompt = f"""
{new_mutation}INSTRUCTION: {top_response}
INSTRUCTION MUTANT ="""

# Retry wrapper
def generate_with_retry(prompt, base_delay=2):
    attempt = 0
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200
            )
            # response = model.generate_content(prompt)
            # return response.text.strip()
            return response
        except GoogleAPIError as e:
            wait = base_delay * (2 ** attempt)
            print(f"[Retry {attempt + 1}] Error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            attempt += 1

# Generate and save responses
responses = {}
for i in range(5):
    print(f"Generating response {i+1}...")

    generated_text = generate_with_retry(gemini_prompt)

    # simplified_output = model.generate_content(generated_text + "\n" + test_complex).text.strip()
    new_prompt = generated_text.choices[0].message.content.strip() + "\n" + test_complex
    simplified_output = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": new_prompt}],
        max_tokens=200
    )
    simplified_output = simplified_output.choices[0].message.content.strip()
    reward = score_output(test_complex, simplified_output)

    responses[str(i)] = {
        "gemini_prompt": gemini_prompt,
        "mutation_prompt": new_mutation,
        "response": generated_text.choices[0].message.content.strip(),
        "simplified_output": simplified_output,
        "test_sentence": test_complex,
        "reward": reward
    }

    print(f"[{i}] reward = {reward:.2f}\n")

# Save all responses
with open("gemini_responses_gen2.json", "w") as f:
    json.dump(responses, f, indent=2)

Generating response 1...
[0] reward = 31.85

Generating response 2...
[1] reward = 31.41

Generating response 3...
[2] reward = 37.02

Generating response 4...
[3] reward = 33.46

Generating response 5...
[4] reward = 32.88



In [ ]:
with open("gemini_responses_gen2.json") as f:
    data2 = json.load(f)

sorted_entries = sorted(data2.items(), key=lambda x: x[1]["reward"], reverse=True)

top_response = sorted_entries[0][1]["response"]

new_mutation = zero_order_hyper_mutation(sorted_entries)

gemini_prompt = f"""
{new_mutation}INSTRUCTION: {top_response}
INSTRUCTION MUTANT ="""

# Retry wrapper
def generate_with_retry(prompt, base_delay=2):
    attempt = 0
    while True:
        try:
            # response = model.generate_content(prompt)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200
            )
            # return response.text.strip()
            return response
        except GoogleAPIError as e:
            wait = base_delay * (2 ** attempt)
            print(f"[Retry {attempt + 1}] Error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            attempt += 1

# Generate and save responses
responses = {}
for i in range(5):
    print(f"Generating response {i+1}...")

    generated_text = generate_with_retry(gemini_prompt)

    new_prompt = generated_text.choices[0].message.content.strip() + "\n" + test_complex

    simplified_output = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": new_prompt}],
        max_tokens=200
    )
    simplified_output = simplified_output.choices[0].message.content.strip()
    reward = score_output(test_complex, simplified_output)

    responses[str(i)] = {
        "gemini_prompt": gemini_prompt,
        "mutation_prompt": new_mutation,
        "response": generated_text.choices[0].message.content.strip(),
        "simplified_output": simplified_output,
        "test_sentence": test_complex,
        "reward": reward
    }

    print(f"[{i}] reward = {reward:.2f}\n")

# Save all responses
with open("gemini_responses_gen3.json", "w") as f:
    json.dump(responses, f, indent=2)

Generating response 1...
[0] reward = 29.10

Generating response 2...
[1] reward = 31.59

Generating response 3...
[2] reward = 33.33

Generating response 4...
[3] reward = 22.81

Generating response 5...
[4] reward = 33.66



In [ ]:
with open("gemini_responses_gen3.json") as f:
    data3 = json.load(f)

sorted_entries = sorted(data3.items(), key=lambda x: x[1]["reward"], reverse=True)

top_response = sorted_entries[0][1]["response"]

new_mutation = zero_order_hyper_mutation(sorted_entries)

gemini_prompt = f"""
{new_mutation}INSTRUCTION: {top_response}
INSTRUCTION MUTANT ="""

# Retry wrapper
def generate_with_retry(prompt, base_delay=2):
    attempt = 0
    while True:
        try:
            # response = model.generate_content(prompt)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200
            )
            # return response.text.strip()
            return response
        except GoogleAPIError as e:
            wait = base_delay * (2 ** attempt)
            print(f"[Retry {attempt + 1}] Error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            attempt += 1

# Generate and save responses
responses = {}
for i in range(5):
    print(f"Generating response {i+1}...")

    generated_text = generate_with_retry(gemini_prompt)

    # simplified_output = model.generate_content(generated_text + "\n" + test_complex).text.strip()
    new_prompt = generated_text.choices[0].message.content.strip() + "\n" + test_complex

    simplified_output = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": new_prompt}],
        max_tokens=200
    )
    simplified_output = simplified_output.choices[0].message.content.strip()
    reward = score_output(test_complex, simplified_output)

    responses[str(i)] = {
        "gemini_prompt": gemini_prompt,
        "mutation_prompt": new_mutation,
        "response": generated_text.choices[0].message.content.strip(),
        "simplified_output": simplified_output,
        "test_sentence": test_complex,
        "reward": reward
    }

    print(f"[{i}] reward = {reward:.2f}\n")

# Save all responses
with open("gemini_responses_gen4.json", "w") as f:
    json.dump(responses, f, indent=2)

Generating response 1...
[0] reward = 34.99

Generating response 2...
[1] reward = 32.17

Generating response 3...
[2] reward = 32.20

Generating response 4...
[3] reward = 40.33

Generating response 5...
[4] reward = 37.66

